In [57]:
import os
import glob
import numpy as np

In [58]:
data_path = './data'
train_data_path = os.path.join(data_path, 'Train')
java_folder_path = train_data_path + '/*.java'
        
file_pair_list = []

for _, folder_name in enumerate(glob.glob(train_data_path + '/*/')):
    file_pair = []
    for _, file_name in enumerate(glob.glob(folder_name + '/*.java')):
        file_pair.append(file_name)
    
    file_pair_list.append(file_pair)

file1_string = ""
with open("./data/Train/0f6ca337_6843cc19/0f6ca337.java", 'r') as file1:
    file1_string = file1.read().replace('\n', '')
    print(file1_string)

file2_string = ""
with open("./data/Train/0f6ca337_6843cc19/6843cc19.java", 'r') as file2:
    file2_string = file2.read().replace('\n', '')
    print(file2_string)

import java.io.*;import java.util.*; import java.math.*;import java.awt.Point; public class Main {    static final long MOD = 1000000007L;    //static final long MOD2 = 1000000009L;    //static final long MOD = 998244353L;    //static final long INF = 500000000000L;    static final int INF =   1000000005;    static final int NINF = -1000000005;    //static final long INF = 1000000000000000000L;    static FastScanner sc;    static PrintWriter pw;    static final int[][] dirs = {{-1,0},{1,0},{0,-1},{0,1}};     public static void main(String[] args) {        sc = new FastScanner();        pw = new PrintWriter(System.out);          int Q = sc.ni();        for (int q = 0; q < Q; q++) {            int N = sc.ni();            int[] nums = sc.intArray(N,0);            Integer[] less = new Integer[N];            Integer[] more = new Integer[N];            TreeSet<Integer> ts = new TreeSet<Integer>();            for (int i = 0; i < N; i++) {                ts.add(nums[i]);                less[i]

In [59]:
"""def read_tokens(text, bag, bag_aux):
    
    text = text + " "
    word_amount = {}
    words_full = bag
    word = ""

    for letter in text:
        if word not in word_amount:
            word_amount[word] = 1
        else: 
            word_amount[word] += 1

        if letter not in bag_aux:
            word += letter.lower()
        elif word != '':
            words_full.append(word)
            word = ''        
        else:
            word = ""
    return words_full, word_amount"""

'def read_tokens(text, bag, bag_aux):\n    \n    text = text + " "\n    word_amount = {}\n    words_full = bag\n    word = ""\n\n    for letter in text:\n        if word not in word_amount:\n            word_amount[word] = 1\n        else: \n            word_amount[word] += 1\n\n        if letter not in bag_aux:\n            word += letter.lower()\n        elif word != \'\':\n            words_full.append(word)\n            word = \'\'        \n        else:\n            word = ""\n    return words_full, word_amount'

In [60]:
def getNgrams(n, text,allNgrams):
    nGrams = {}
    for i in range(len(text)-n+1):
        current = text[i]
        for j in range(i+1,n+i):
            current += " " + (text[j])
        if current not in allNgrams:
            allNgrams.append(current)
        if current not in nGrams:
            nGrams[current] = 1
        else:
            nGrams[current] += 1
    return nGrams,allNgrams


In [61]:
def getVector(word_list, full_word_list):
    vector = []
    for word in full_word_list:
        if word not in word_list:
            vector.append(0)
        else:
            vector.append(word_list[word])
    return vector

In [62]:
"""%run "Lexer.py"
import sys
from Lexer import *

def getTokens(filepath):

    token_words = []

    lexer = Lexer(filepath)
    token = lexer.scan()
    while token.getTag() != Tag.EOF:
        token_words.append(str(token))
        token = lexer.scan()
    token_words.append("END")

    return token_words"""


'%run "Lexer.py"\nimport sys\nfrom Lexer import *\n\ndef getTokens(filepath):\n\n    token_words = []\n\n    lexer = Lexer(filepath)\n    token = lexer.scan()\n    while token.getTag() != Tag.EOF:\n        token_words.append(str(token))\n        token = lexer.scan()\n    token_words.append("END")\n\n    return token_words'

In [63]:
def cosineSimilarity(word_vectors):
    
    for i in range(len(word_vectors)):
        word_vectors[i] = np.array(word_vectors[i])
        
    numerator = word_vectors[0]

    for i in range(1, len(word_vectors)):
        numerator = numerator * word_vectors[i]
    
    numerator = np.sum(numerator)

    denominator = 1

    for i in range(len(word_vectors)):
        denominator *= np.sqrt(np.sum(word_vectors[i] ** 2))

    return numerator / denominator

In [64]:
import javalang

def getTokens (file):
    token_words = []

    tokens = list(javalang.tokenizer.tokenize(file))
    parser = javalang.parser.Parser(tokens)
    
    for i in tokens:
       token_words.append(type(i).__name__)
    
    return token_words

In [65]:
def get_Prediction (file1_string, file2_string):

    allNgrams = []

    token_words1 = getTokens(file1_string)
    token_words2 = getTokens(file2_string)
    nGrams1,allNgrams = getNgrams(3,token_words1,allNgrams)
    nGrams2,allNgrams = getNgrams(3,token_words2,allNgrams)

    vector1 = getVector(nGrams1,allNgrams)
    vector2 = getVector(nGrams2,allNgrams)

    print("1: ")
    print(nGrams1)
    print("2: ") 
    print(nGrams2)
    print("ALL: ")
    print(allNgrams)

    print("vector1:")
    print(vector1)
    print("vector2:")
    print(vector2)


    return(cosineSimilarity([vector1, vector2]))

1: 
{'Keyword Identifier Separator': 5, 'Identifier Separator Identifier': 5, 'Separator Identifier Separator': 5, 'Identifier Separator Operator': 3, 'Separator Operator Separator': 3, 'Operator Separator Keyword': 3, 'Separator Keyword Identifier': 3, 'Identifier Separator Modifier': 2, 'Separator Modifier Keyword': 1, 'Modifier Keyword Identifier': 1, 'Separator Modifier Modifier': 1, 'Modifier Modifier BasicType': 1, 'Modifier BasicType Identifier': 1, 'BasicType Identifier Operator': 1, 'Identifier Operator DecimalInteger': 1, 'Operator DecimalInteger Separator': 1}
2: 
{'Keyword Identifier Separator': 5, 'Identifier Separator Identifier': 5, 'Separator Identifier Separator': 5, 'Identifier Separator Operator': 3, 'Separator Operator Separator': 3, 'Operator Separator Keyword': 3, 'Separator Keyword Identifier': 3, 'Identifier Separator Modifier': 2, 'Separator Modifier Keyword': 1, 'Modifier Keyword Identifier': 1, 'Separator Modifier Modifier': 1, 'Modifier Modifier BasicType': 